# Homework 2

## Import packages and data

Import necessary packages.

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import math

import wrds
import fredapi

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.formula.api import ols
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from matplotlib import pyplot as plt

Import data from WRDS: S&P 500 returns, dividends, levels.

In [2]:
wrds_conn = wrds.Connection(wrds_username='wmann')
data = wrds_conn.raw_sql("select caldt, vwretd, vwretx, spindx, totval from crsp_a_indexes.msp500")
wrds_conn.close() 

data['month'] = pd.to_datetime(data['caldt']).dt.to_period('M')
data = data.set_index(['month']).drop('caldt',axis=1)

# Calculate S&P 500 dividends as the difference between monthly return with and without dividends.
data['dividends_mth'] = ( data['vwretd'] - data['vwretx'] )*data['spindx'].shift()

# Aggregate dataset to quarterly level.
dataq = data.reset_index()
dataq['quarter'] = dataq['month'].dt.to_timestamp().dt.to_period('Q')
dataq = dataq.set_index('quarter')
dataq['dividends_qtr'] = dataq.groupby(level=0)['dividends_mth'].sum()
dataq = dataq.groupby(level=0).last()

# dataq = dataq['1926Q1':]
dataq = dataq['2000':]

Loading library list...
Done


Calculate the series that we will actually use in the analysis

In [3]:
log_spindx = np.log(dataq['spindx'])
log_spindx_growth = log_spindx.diff().dropna()

log_dividends = np.log(dataq['dividends_qtr'])
log_dividends_growth = log_dividends.diff().dropna()

log_DP = np.log(dataq['dividends_qtr']/dataq['spindx'])

## Assignment

The code above calculates five different series, at a quarterly frequency, from 2000 to the present:
1. The S&P 500 index level, measured as a logarithm.
2. The change in the above value (that is, the percentage increase in the S&P 500 index).
3. The dividends paid by S&P 500 stocks, as a logarithm.
4. The change in the above value (that is, the percentage growth of S&P 500 dividend payments).
5. The S&P 500 dividend yield (dividends as a fraction of index value), as a logarithm.

For each of the above series, plot them, and then conduct an ADF test and a KPSS test (allowing statsmodels to choose the optimal number of lags). What, if anything, do these tests tell you about whether it is appropriate to model the series as stationary?

Finally, for the last ADF test that you ran (on the dividend yield), show how the test statistic from this test can be located in the output from a regression with the statsmodels summary() command. Then explain why the p-value from the test is *not* located anywhere in the same regression output.